In [ ]:
# Expectation to previous code

In [ ]:
#liabries installed:        !pip install chromadb Pillow matplotlib numpy
#imports:
                            #import chromadb
                            #import numpy as np
                            #from PIL import Image
                            #import matplotlib.pyplot as plt
                            #import os
                            #from sentence_transformers import SentenceTransformer
                            #model = SentenceTransformer('clip-ViT-B-32')

#instantiate the model:     client = chromadb.Client()

In [3]:
#Establish ChromaDB connection and load the client

In [ ]:
# Load environment variables from .env file

#load_dotenv() #we assume that a .env file has been created that stores that API credentials (shared in slack channel)

# Get the API key and other details from environment variables

#api_key = os.getenv("CHROMA_API_KEY")
#tenant_id = os.getenv("CHROMA_TENANT")
#database_name = os.getenv("CHROMA_DATABASE")

#load client
#client = chromadb.CloudClient(
        #api_key=api_key,
        #tenant=tenant_id,
        #database=database_name
    #)

In [2]:
import os
# TODO: update path to use os
import PIL.Image as Image

from sentence_transformers import SentenceTransformer

import chromadb
from PIL import Image

/home/gwenm/.pyenv/versions/3.10.6/envs/inspiart/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-09-10 12:02:25.833595: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [9]:
model = SentenceTransformer('clip-ViT-B-32')
chroma_client = chromadb.CloudClient(
        api_key=os.environ.get("CHROMA_API_KEY"),
        tenant=os.environ.get("CHROMA_TENANT"),
        database='inspiart'
        )
collection = chroma_client.get_or_create_collection(name="wikiart_115000images")

In [56]:
from tensorflow.keras import models

model_keras =  models.load_model("../models/model_Xception_alldata_finetuned.keras")

I0000 00:00:1757504456.216453   35032 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1303 MB memory:  -> device: 0, name: Quadro P520, pci bus id: 0000:6c:00.0, compute capability: 6.1


In [ ]:
def get_similar_images(image_query: Image.Image, collection, model, n_results: int = 5):
    """
    Finds the 5 most similar images in the collection for a given image query and
    handles the case where the query image itself is in the database.
    """
    # 1. Encode the input image to get its embedding
    query_embedding = model.encode(image_query, device="cpu").tolist()

    # 2. Query for 5 results + 1 to handle the case where the query image is the top match
    results = collection.query(
        query_embeddings=[query_embedding],
        n_results=n_results + 1,
        include=['metadatas', 'distances']
    )

    # 3. Process the results
    metadatas = results['metadatas'][0]
    distances = results['distances'][0]

    # Check if the most similar image is the image itself which means the distance is arbitrarilly close to 0
    is_exact_match = distances[0] < 0.000001

    # If it's an exact match, we return its metadata and the next 5 most similar images, otherwise, we just return the top 5.
    if is_exact_match:
        # The first result is the image itself, so we return its metadata.
        exact_match_data = {
            "artist": metadatas[0].get('artist'),
            "artwork_name": metadatas[0].get('document'),
        }
        similar_images_metadata = metadatas[1:]
    else:
        # The image is not in our dataset, so there is no exact match.
        exact_match_data = None
        similar_images_metadata = metadatas[:n_results]

   # 4. Format the list of similar images for the final output
    similar_images_output = []
    for meta in similar_images_metadata:
        similar_images_output.append({
            "artist": meta.get('artist'),
            "artwork_name": meta.get('document'),
        })

    # 5. Return a structured dictionary
    return {
        "query_image_found_in_db": is_exact_match,
        "exact_match_details": exact_match_data,
        "similar_images": similar_images_output
    }

In [14]:
path = "../raw_data/Images to try/Piet_Mondriaan,_1942_-_New_York_City_I.jpg"
img = Image.open(path)
get_similar_images(img, collection, model, n_results = 5)

{'query_image_found_in_db': False,
 'exact_match_details': None,
 'similar_images': [{'artist': 'Piet Mondrian', 'artwork_name': None},
  {'artist': 'Piet Mondrian', 'artwork_name': None},
  {'artist': 'Piet Mondrian', 'artwork_name': None},
  {'artist': 'Richard Paul Lohse', 'artwork_name': None},
  {'artist': 'Theo van Doesburg', 'artwork_name': None}]}

In [37]:
def receive_image(path):

    #get the image from the POST request

    #contents = img.file.read()

    working_image = Image.open(path)

    #get or create a connection

    images_db = chroma_client.get_or_create_collection(name="wikiart_115000images")

    # Use the CLIP model to encode the image

    query_embedding = model.encode(working_image, device="cpu").tolist()

    #perform the query

    image_suggestions = images_db.query(
    query_embeddings=[query_embedding],
    include=['uris','metadatas', 'distances'],
    n_results=3
    )

    # Search if the image is matches with the first result as an output:
    distances = image_suggestions['distances'][0]

    print(image_suggestions['metadatas'][0][0])
    print(distances[0])
    print(image_suggestions['metadatas'][0][1])
    print(distances[1])
    print(image_suggestions['metadatas'][0][2])
    print(distances[2])

    return None
    is_exact_match = distances[0] < 0.000001

    # if it matches, return its infos, else return we don't know the image
    if is_exact_match:
        image_dict = {
            "artist" : image_suggestions['metadatas'][0][0]['artist'],
            "file_name" : image_suggestions['metadatas'][0][0]['file_name']
            }
    else :
        image_dict = {
            "artist" : "Unknown artist",
            "file_name" : "Unknown artwork"
            }

    return image_dict


In [38]:
path = "/home/gwenm/code/gwen-m97/inspiart/raw_data/wiki/new-york-city-i-1942.jpg!Large.jpg"
receive_image(path)

{'wikiart_url': 'https://www.wikiart.org/en/piet-mondrian/new-york-city-i-1942', 'style': 'Neoplasticism', 'img_url': 'https://uploads5.wikiart.org/images/piet-mondrian/new-york-city-i-1942.jpg!Large.jpg', 'artist': 'Piet Mondrian', 'movement': 'Neo-Impressionism', 'file_name': '112511-new-york-city-i-1942.jpg'}
5.4051226e-09
{'file_name': '112513-place-de-la-concorde-1943.jpg', 'movement': 'Neo-Impressionism', 'style': 'Neoplasticism', 'img_url': 'https://uploads1.wikiart.org/images/piet-mondrian/place-de-la-concorde-1943.jpg!Large.jpg', 'wikiart_url': 'https://www.wikiart.org/en/piet-mondrian/place-de-la-concorde-1943', 'artist': 'Piet Mondrian'}
19.593811
{'img_url': 'https://uploads7.wikiart.org/images/richard-paul-lohse/rhythmische-progression-drei-gleicher-farbgruppen-1952.jpg', 'file_name': '171310-rhythmische-progression-drei-gleicher-farbgruppen-1952.jpg', 'style': 'Concretism', 'movement': 'Constructivism', 'wikiart_url': 'https://www.wikiart.org/en/richard-paul-lohse/rhythmi

In [43]:
path = "/home/gwenm/code/gwen-m97/inspiart/raw_data/Images to try/Piet_Mondriaan,_1942_-_New_York_City_I.jpg"
receive_image(path)

{'artist': 'Piet Mondrian', 'img_url': 'https://uploads5.wikiart.org/images/piet-mondrian/new-york-city-i-1942.jpg!Large.jpg', 'file_name': '112511-new-york-city-i-1942.jpg', 'style': 'Neoplasticism', 'movement': 'Neo-Impressionism', 'wikiart_url': 'https://www.wikiart.org/en/piet-mondrian/new-york-city-i-1942'}
13.06532
{'file_name': '112513-place-de-la-concorde-1943.jpg', 'style': 'Neoplasticism', 'wikiart_url': 'https://www.wikiart.org/en/piet-mondrian/place-de-la-concorde-1943', 'img_url': 'https://uploads1.wikiart.org/images/piet-mondrian/place-de-la-concorde-1943.jpg!Large.jpg', 'movement': 'Neo-Impressionism', 'artist': 'Piet Mondrian'}
19.908276
{'movement': 'Neo-Impressionism', 'img_url': 'https://uploads3.wikiart.org/images/piet-mondrian/broadway-boogie-woogie-1943.jpg!Large.jpg', 'artist': 'Piet Mondrian', 'wikiart_url': 'https://www.wikiart.org/en/piet-mondrian/broadway-boogie-woogie-1943', 'style': 'Unknown', 'file_name': '112512-broadway-boogie-woogie-1943.jpg'}
21.477737

In [39]:
path = "/home/gwenm/code/gwen-m97/inspiart/raw_data/wiki/water-lilies-nympheas-1907.jpg!Large.jpg"
receive_image(path)

{'file_name': '93941-water-lilies-nympheas-1907.jpg', 'style': 'Impressionism', 'img_url': 'https://uploads1.wikiart.org/images/claude-monet/water-lilies-nympheas-1907.jpg!Large.jpg', 'wikiart_url': 'https://www.wikiart.org/en/claude-monet/water-lilies-nympheas-1907', 'movement': 'Impressionism', 'artist': 'Claude Monet'}
1.8473374e-10
{'artist': 'Claude Monet', 'style': 'Impressionism', 'wikiart_url': 'https://www.wikiart.org/en/claude-monet/water-lilies-25', 'movement': 'Impressionism', 'img_url': 'https://uploads8.wikiart.org/images/claude-monet/water-lilies-25.jpg!Large.jpg', 'file_name': '93937-water-lilies-25.jpg'}
10.763243
{'style': 'Impressionism', 'img_url': 'https://uploads8.wikiart.org/images/claude-monet/water-lilies-30.jpg!Large.jpg', 'artist': 'Claude Monet', 'file_name': '93977-water-lilies-30.jpg', 'movement': 'Impressionism', 'wikiart_url': 'https://www.wikiart.org/en/claude-monet/water-lilies-30'}
12.751024


In [41]:
path = "/home/gwenm/code/gwen-m97/inspiart/raw_data/wiki/old-fashioned-vegetable.jpg"
receive_image(path)

{'file_name': '142654-serenade-du-pave-1892-1.jpg', 'wikiart_url': 'https://www.wikiart.org/en/theophile-steinlen/serenade-du-pave-1892-1', 'movement': 'Art Nouveau', 'style': 'Art Nouveau (Modern)', 'img_url': 'https://uploads4.wikiart.org/images/theophile-steinlen/serenade-du-pave-1892-1.jpg!Large.jpg', 'artist': 'Theophile Steinlen'}
81.67352
{'style': 'Dada', 'artist': 'Marcel Duchamp', 'file_name': '169190-apolinere-enamelled-1916.jpg', 'wikiart_url': 'https://www.wikiart.org/en/marcel-duchamp/apolinere-enamelled-1916', 'movement': 'Dada', 'img_url': 'https://uploads1.wikiart.org/images/marcel-duchamp/apolinere-enamelled-1916.jpg!Large.jpg'}
83.426735
{'style': 'Social Realism', 'movement': 'Social Realism', 'file_name': '140790-say-no.jpg', 'wikiart_url': 'https://www.wikiart.org/en/sue-coe/say-no', 'img_url': 'https://uploads2.wikiart.org/images/sue-coe/say-no.jpg', 'artist': 'Sue Coe'}
87.94089


In [45]:
path = "/home/gwenm/code/gwen-m97/inspiart/raw_data/wiki/lpdp_25399-1.jpg"
receive_image(path)

{'artist': 'Paul Cezanne', 'file_name': '113069-portrait-of-ambroise-vollard-1899.jpg', 'wikiart_url': 'https://www.wikiart.org/en/paul-cezanne/portrait-of-ambroise-vollard-1899', 'img_url': 'https://uploads8.wikiart.org/images/paul-cezanne/portrait-of-ambroise-vollard-1899.jpg!Large.jpg', 'style': 'Post-Impressionism', 'movement': 'Post-Impressionism'}
8.258798
{'movement': 'Symbolism', 'img_url': 'https://uploads1.wikiart.org/images/ferdinand-hodler/self-portrait.jpg!Large.jpg', 'artist': 'Ferdinand Hodler', 'wikiart_url': 'https://www.wikiart.org/en/ferdinand-hodler/self-portrait', 'file_name': '81729-self-portrait.jpg', 'style': 'Art Nouveau (Modern)'}
20.45548
{'style': 'Post-Impressionism', 'movement': 'Post-Impressionism', 'wikiart_url': 'https://www.wikiart.org/en/pyotr-konchalovsky/portrait-of-the-sculptor-p-bromirsky-1919', 'artist': 'Pyotr Konchalovsky', 'file_name': '120892-portrait-of-the-sculptor-p-bromirsky-1919.jpg', 'img_url': 'https://uploads3.wikiart.org/images/pyotr

In [46]:
path = "/home/gwenm/code/gwen-m97/inspiart/raw_data/Images to try/Van_Gogh_-_Starry_Night_-_Google_Art_Project.jpg"
receive_image(path)

{'img_url': 'https://uploads4.wikiart.org/00142/images/vincent-van-gogh/the-starry-night.jpg!Large.jpg', 'artist': 'Vincent van Gogh', 'wikiart_url': 'https://www.wikiart.org/en/vincent-van-gogh/the-starry-night-1889', 'style': 'Unknown', 'movement': 'Post-Impressionism', 'file_name': '115385-the-starry-night.jpg'}
3.1712368
{'artist': 'Vincent van Gogh', 'file_name': '115176-starry-night-over-the-rhone.jpg', 'wikiart_url': 'https://www.wikiart.org/en/vincent-van-gogh/the-starry-night-1888-2', 'movement': 'Post-Impressionism', 'style': 'Post-Impressionism', 'img_url': 'https://uploads0.wikiart.org/00175/images/vincent-van-gogh/starry-night-over-the-rhone.jpg!Large.jpg'}
23.794777
{'wikiart_url': 'https://www.wikiart.org/en/vincent-van-gogh/road-with-cypresses-1890', 'img_url': 'https://uploads3.wikiart.org/images/vincent-van-gogh/road-with-cypresses-1890.jpg!Large.jpg', 'style': 'Post-Impressionism', 'artist': 'Vincent van Gogh', 'file_name': '115534-road-with-cypresses-1890.jpg', 'mov

In [ ]:
for i in range(1,6) :
    print(i)

1
2
3
4
5


In [49]:
def test_image1(path):

    #get the image from the POST request

    #contents = img.file.read()

    working_image = Image.open(path)

    #get or create a connection

    images_db = chroma_client.get_or_create_collection(name="wikiart_115000images")

    # Use the CLIP model to encode the image

    query_embedding = model.encode(working_image, device="cpu").tolist()

    #perform the query

    image_suggestions = images_db.query(
    query_embeddings=[query_embedding],
    include=['uris','metadatas', 'distances'],
    n_results=3
    )

    # Search if the image is matches with the first result as an output:
    distances = image_suggestions['distances'][0]

    is_exact_match = distances[0] < 10

    # if it matches, return its infos, else return we don't know the image
    if is_exact_match:
        image_dict = {
            "artist" : image_suggestions['metadatas'][0][0]['artist'],
            "file_name" : image_suggestions['metadatas'][0][0]['file_name']
            }
    else :
        image_dict = {
            "artist" : "Unknown artist",
            "file_name" : "Unknown artwork"
            }

    return image_dict


In [50]:
#TEST WITH IMAGE IN
path = "/home/gwenm/code/gwen-m97/inspiart/raw_data/wiki/lpdp_25399-1.jpg"
test_image1(path)

{'artist': 'Paul Cezanne',
 'file_name': '113069-portrait-of-ambroise-vollard-1899.jpg'}

In [51]:
#TEST WITHOUT IMAGE IN
path = "/home/gwenm/code/gwen-m97/inspiart/raw_data/wiki/old-fashioned-vegetable.jpg"
test_image1(path)

{'artist': 'Unknown artist', 'file_name': 'Unknown artwork'}

In [65]:
def test_image2(path):

    #get the image from the POST request

    #contents = img.file.read()

    working_image = Image.open(path)

  #styles constant

    LIST_STYLES = ['Abstract Art', 'Abstract Expressionism', 'Academicism', 'Art Deco', 'Art Informel', 'Art Nouveau (Modern)', 'Biedermeier', 'Color Field Painting', 'Conceptual Art', 'Concretism', 'Contemporary', 'Contemporary Realism', 'Cubism', 'Dada', 'Divisionism', 'Expressionism', 'Fantastic Realism', 'Fauvism', 'Figurative Expressionism', 'Futurism', 'Hard Edge Painting', 'Hyper-Realism', 'Impressionism', 'Kitsch', 'Luminism', 'Lyrical Abstraction', 'Magic Realism', 'Metaphysical art', 'Minimalism', 'Native Art', 'Naturalism', 'Naïve Art (Primitivism)', 'Neo-Dada', 'Neo-Expressionism', 'Neo-Impressionism', 'Neo-Pop Art', 'Neo-Romanticism', 'Neoclassicism', 'New European Painting', 'Op Art', 'Orientalism', 'Pop Art', 'Post-Impressionism', 'Post-Painterly Abstraction', 'Precisionism', 'Realism', 'Regionalism', 'Romanticism', 'Social Realism', 'Socialist Realism', 'Surrealism', 'Symbolism', 'Synthetic Cubism', 'Tachisme', 'Tonalism', 'Transavantgarde']

    #going to find the style

    #PREPROCESSING

    img = working_image.convert('RGB')
    img_resized = img.resize((224, 224), Image.BICUBIC)
    img_array = np.array(img_resized)
    img_preprocessed = preprocess_input(img_array)
    img_batch = np.expand_dims(img_preprocessed, axis=0)  # shape (1,224,224,3)

    #PREDICTION

    preds = model_keras.predict(img_batch)
    pred_indice = preds.argmax(axis=1)[0] #Take the number
    style_predicted = LIST_STYLES[pred_indice]

    #GET IMAGES THAT MATCH WITH STYLE AND IMAGE

    #get or create a connection

    images_db = chroma_client.get_or_create_collection(name="wikiart_115000images")

    # Use the CLIP model to encode the image

    query_embedding = model.encode(working_image, device="cpu").tolist()

    #perform the query

    image_suggestions = images_db.query(
    query_embeddings=[query_embedding],
    include=['uris','metadatas', 'distances'],
    n_results=6
    )

    # Search if the image is matches with the first result as an output:
    distances = image_suggestions['distances'][0]

    is_exact_match = distances[0] < 10

    #create a dictionary of the results

    image_dict = {}

    if is_exact_match:
        for i in range(1,6) :
            key=f"image_{i-1}"
            image_dict[key] = {
                "img_url" : image_suggestions['metadatas'][0][i]['img_url'],
                "artist" : image_suggestions['metadatas'][0][i]['artist'],
                "file_name" : image_suggestions['metadatas'][0][i]['file_name'],
                "style" : image_suggestions['metadatas'][0][i]['style']
                }
    else :
        for i in range(5) :
            key=f"image_{i}"
            image_dict[key] = {
                "img_url" : image_suggestions['metadatas'][0][i]['img_url'],
                "artist" : image_suggestions['metadatas'][0][i]['artist'],
                "file_name" : image_suggestions['metadatas'][0][i]['file_name'],
                "style" : image_suggestions['metadatas'][0][i]['style']
                }


    final_dict = {"style_predicted" : style_predicted, "images" : image_dict}

    #return the dictionary

    return final_dict

In [66]:
#TEST WITH IMAGE IN
#Shouldn't return artist': 'Paul Cezanne', 'file_name': '113069-portrait-of-ambroise-vollard-1899.jpg',
path = "/home/gwenm/code/gwen-m97/inspiart/raw_data/wiki/lpdp_25399-1.jpg"
test_image2(path)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 145ms/step


{'style_predicted': 'Post-Impressionism',
 'images': {'image_0': {'img_url': 'https://uploads1.wikiart.org/images/ferdinand-hodler/self-portrait.jpg!Large.jpg',
   'artist': 'Ferdinand Hodler',
   'file_name': '81729-self-portrait.jpg',
   'style': 'Art Nouveau (Modern)'},
  'image_1': {'img_url': 'https://uploads3.wikiart.org/images/pyotr-konchalovsky/portrait-of-the-sculptor-p-bromirsky-1919.jpg',
   'artist': 'Pyotr Konchalovsky',
   'file_name': '120892-portrait-of-the-sculptor-p-bromirsky-1919.jpg',
   'style': 'Post-Impressionism'},
  'image_2': {'img_url': 'https://uploads1.wikiart.org/images/suzanne-valadon/portrait-of-maurice-utrillo-1921.jpg!Large.jpg',
   'artist': 'Suzanne Valadon',
   'file_name': '117611-portrait-of-maurice-utrillo-1921.jpg',
   'style': 'Post-Impressionism'},
  'image_3': {'img_url': 'https://uploads1.wikiart.org/images/frederic-bazille/portrait-of-auguste-renoir-1867.jpg!Large.jpg',
   'artist': 'Frederic Bazille',
   'file_name': '95913-portrait-of-aug

In [ ]:
#TEST WITHOUT IMAGE IN
# should return first Theophile Steinlen cause no filtering
path = "/home/gwenm/code/gwen-m97/inspiart/raw_data/wiki/old-fashioned-vegetable.jpg"
test_image2(path)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step


{'style_predicted': 'Pop Art',
 'images': {'image_0': {'img_url': 'https://uploads4.wikiart.org/images/theophile-steinlen/serenade-du-pave-1892-1.jpg!Large.jpg',
   'artist': 'Theophile Steinlen',
   'file_name': '142654-serenade-du-pave-1892-1.jpg',
   'style': 'Art Nouveau (Modern)'},
  'image_1': {'img_url': 'https://uploads1.wikiart.org/images/marcel-duchamp/apolinere-enamelled-1916.jpg!Large.jpg',
   'artist': 'Marcel Duchamp',
   'file_name': '169190-apolinere-enamelled-1916.jpg',
   'style': 'Dada'},
  'image_2': {'img_url': 'https://uploads2.wikiart.org/images/sue-coe/say-no.jpg',
   'artist': 'Sue Coe',
   'file_name': '140790-say-no.jpg',
   'style': 'Social Realism'},
  'image_3': {'img_url': 'https://uploads5.wikiart.org/images/john-bauer/jultomten-1911.jpg!Large.jpg',
   'artist': 'John Bauer',
   'file_name': '52913-jultomten-1911.jpg',
   'style': 'Art Nouveau (Modern)'},
  'image_4': {'img_url': 'https://uploads1.wikiart.org/images/theophile-steinlen/serenade-du-pave-18